In [ ]:
from __future__ import division
from gridder import mk_grid
import coordinates

import sys, os
import numpy as np
from scipy.optimize import curve_fit
from scipy import interpolate as interp
import netCDF4
from netCDF4 import MFDataset
import matplotlib.pyplot as plt

In [ ]:
# see time_map.py for detailed explanation

pi = np.pi

import warnings
warnings.resetwarnings()

def get_data(data_file, mc, version):
    # Extract sync time from MCE header
    nvar = 17
    time_ind = 15
    time_get_inds = [time_ind + i for i in np.arange(100)*nvar]
    mce_sync_time = data_file.variables['mce'+str(mc)+'_header'][:,time_get_inds,0].flatten()

    # Extract counts from MCE data
    mce_counts_all = data_file.variables['mce'+str(mc)+'_raw_data_all'][:]

    data = np.zeros((16,60,len(mce_sync_time)))
    for x in range(16):
        for f in range(60):
            cr = coordinates.xf_to_muxcr(x,f,mc)
            data[x,f,:] = mce_counts_all[:,cr[1],cr[0],:].flatten()

    # Extract time and pointing from telescope data
    time_ind = 20
    ra_ind = 9
    dec_ind = 10
    flag_ind = 1

    telescope_time = data_file.variables['tel'][:,:,time_ind].flatten()
    telescope_ra = data_file.variables['tel'][:,:,ra_ind].flatten()
    telescope_dec = data_file.variables['tel'][:,:,dec_ind].flatten()
    telescope_flag = data_file.variables['tel'][:,:,flag_ind].flatten()

    # Generate pointing spline for telescope
    telescope_data = np.squeeze(np.dstack((telescope_time, telescope_ra, telescope_dec, telescope_flag)))
    telescope_data = telescope_data[telescope_data[:,0]<1e10]
    telescope_data = telescope_data[np.argsort(telescope_data[:,0])]

    ra_spline = interp.UnivariateSpline(telescope_data[:,0], telescope_data[:,1], s=0, ext=3)
    dec_spline = interp.UnivariateSpline(telescope_data[:,0], telescope_data[:,2], s=0, ext=3)

    # Get mapping from sync time to UTC (to match telescope)
    if version == 'correct':
        mce_unix_time = data_file.variables['time'][:,:,0].flatten()
    else:
        times = data_file.variables['time'][:]
        times = np.sort(times, axis=0)
        sync_time_spline = interp.UnivariateSpline(times[:,1], times[:,0], s=0)
        mce_unix_time = sync_time_spline(mce_sync_time)

    # Map MCE time to RA/Dec
    ra = ra_spline(mce_unix_time)
    dec = dec_spline(mce_unix_time)
    values = data

    # Determine telescope state
    flag_spline = interp.UnivariateSpline(telescope_data[:,0], telescope_data[:,3], s=0, ext=3)
    flag = flag_spline(mce_unix_time)
    flag = np.around(flag)

    # sort everything by time
    sorter = np.argsort(mce_unix_time)

    return mce_unix_time[sorter], ra[sorter], dec[sorter], values[:,:,sorter], flag[sorter].astype('int')

# class to hold data from telescope and produce fit
class dataset:
    def __init__(self, path, mc=0, version='correct'):
        # Check inputs
        if mc not in [0,1]:
            print('invalid mc')
            return

        # Load data
        if os.path.exists(path):
            files = [path + i for i in os.listdir(path) if i.endswith(".nc")]
            files = sorted(files, key = os.path.getctime)

            self.t, self.ra, self.dec, self.data, self.flags = get_data(MFDataset(files), mc, version)
        else:
            print('file not found')
            return

        self.scan_flags = np.ones(self.flags.shape, dtype='int')

        # Store a second copy of data that won't be modified
        self.t_full = np.copy(self.t)
        self.ra_full = np.copy(self.ra)
        self.dec_full = np.copy(self.dec)
        self.flags_full = np.copy(self.flags)
        self.scan_flags_full = np.copy(self.scan_flags)
        self.data_full = np.copy(self.data)

    # function to reset data
    def reset(self):
        self.t = np.copy(self.t_full)
        self.ra = np.copy(self.ra_full)
        self.dec = np.copy(self.dec_full)
        self.flags = np.copy(self.flags_full)
        self.scan_flags = np.copy(self.scan_flags_full)
        self.data = np.copy(self.data_full)

    # function to identify scans
    def flag_scans(self, dabs=.001, ddec=.001):
        scan_flags = self.scan_flags

        difs1 = np.abs(self.dec[:-1]-self.dec[1:])
        inds1 = np.where(difs1 > ddec)[0]
        difs2 = np.sqrt((self.ra[:-1]-self.ra[1:])**2 + (self.dec[:-1]-self.dec[1:])**2)
        inds2 = np.where(difs2 > dabs)[0]
        difs3 = np.abs(self.flags[:-1]-self.flags[1:])
        inds3 = np.where(difs3 > .1)[0]
        inds = np.sort(np.unique(np.concatenate((inds1,inds2,inds3))))

        scan_flags[:inds[0]+1] = 1
        for i in range(1,len(inds)-1):
            scan_flags[inds[i]:inds[i+1]] = i
        scan_flags[inds[-1]:] = i

        scan_flags[1:-1][(scan_flags[1:-1] != scan_flags[:-2]) & (scan_flags[1:-1] != scan_flags[2:])] = 0
        self.scan_flags = scan_flags

    # function to remove scan flag == 0
    def remove_scan_flag(self):
        self.t = self.t[self.scan_flags != 0]
        self.ra = self.ra[self.scan_flags != 0]
        self.dec = self.dec[self.scan_flags != 0]
        self.data = self.data[:,:,self.scan_flags != 0]
        self.flags = self.flags[self.scan_flags != 0]
        self.scan_flags = self.scan_flags[self.scan_flags != 0]

    # function to remove anything less than 100 measurements
    def remove_short_scans(self, thresh=100):
        u, n = np.unique(self.scan_flags, return_counts=True)
        u = u[np.where(n >= thresh)]
        self.t = self.t[np.isin(self.scan_flags, u)]
        self.ra = self.ra[np.isin(self.scan_flags, u)]
        self.dec = self.dec[np.isin(self.scan_flags, u)]
        self.data = self.data[:,:,np.isin(self.scan_flags, u)]
        self.flags = self.flags[np.isin(self.scan_flags, u)]
        self.scan_flags = self.scan_flags[np.isin(self.scan_flags, u)]

    # function to remove flags
    def remove_obs_flag(self,flags=[0,1,4]):
        for i in flags:
            self.t = self.t[self.flags != i]
            self.ra = self.ra[self.flags != i]
            self.dec = self.dec[self.flags != i]
            self.data = self.data[:,:,self.flags != i]
            self.scan_flags = self.scan_flags[self.flags != i]
            self.flags = self.flags[self.flags != i]

    # function to fit a polynomial to flags (all detectors)
    def filter_scan(self, n=2):
        for x in range(16):
            for f in range(60):
                for i in np.unique(self.scan_flags):
                    inds = np.where(self.scan_flags==i)[0]
                    fit = np.polyfit(self.t[inds], self.data[x,f,inds], deg=n)
                    self.data[x,f,inds] -= np.polyval(fit,self.t[inds])

    # grid all detectors
    def make_map(self, pixel=1./60./2.):
        pos = np.array([self.ra,self.dec]).T

        vals = np.array([self.data[x,f] for x in range(16) for f in range(60)])
        to_grid = np.concatenate([vals,np.ones((1,len(vals[0])))]).T

        grids, ax = mk_grid(pos, to_grid, calc_density=False, voxel_length=pixel, fit_dims=True)
        self.map_ra = ax[0].flatten() - pixel/2
        self.map_dec = ax[1].flatten() - pixel/2
        self.maps = np.zeros((grids[:,:,0].shape[1],grids[:,:,0].shape[0],16,60))

        grids = grids.transpose((1,0,2))
        self.maps = grids[:,:,:-1].reshape(grids[:,:,0].shape[0],grids[:,:,0].shape[1],16,60)
        self.maps = self.maps.transpose((2,3,0,1))
        self.maps /= grids[:,:,-1]

    # wrapper to do everything
    def reduce_data(self, n=2, pixel=1./60./2., dabs=.001, ddec=.001, thresh=100):
        print('Reduce Data: removing non-scan data...')
        self.remove_obs_flag()
        self.flag_scans(dabs=dabs, ddec=ddec)
        self.remove_scan_flag()
        self.remove_short_scans(thresh=thresh)

        print('Reduce Data: filtering data...')
        self.filter_scan(n=n)

        print('Reduce Data: making maps...')
        self.make_map(pixel=pixel)

    # plots a map (given x and f to plot)
    def plot_map(self,x,f):
        xx,yy = np.meshgrid(self.map_ra,self.map_dec)
        fig = plt.figure(figsize=(8*np.ptp(self.ra)/np.ptp(self.dec), 8))
        ax = fig.add_subplot(111)
        c = ax.pcolor(xx,yy,self.maps[x,f])
        ax.set(xlabel='RA (degrees)', ylabel='Dec (degrees)')
        fig.colorbar(c)
        fig.savefig('map.pdf')
        plt.show()

In [ ]:
### Directory containing venus observation:
dir = '../../Venus_Data/2019-03-15_Venus_2DR/'

In [ ]:
# load the data
print('Venus mapper: loading data...')
venus = dataset(dir)

In [ ]:
# fit data
venus.reduce_data()
# make maps with .5 arcmin pixels
print('Venus mapper: making maps...')
venus.make_map(pixel=1./120)

In [ ]:
plt.ylabel('RA (deg)')
plt.xlabel('Time (min)')
plt.plot((venus.t - venus.t[0]) / 60, venus.ra)
plt.savefig('ra.pdf')

In [ ]:
plt.ylabel('DEC (deg)')
plt.xlabel('Time (min)')
plt.plot((venus.t - venus.t[0]) / 60, venus.dec)
plt.savefig('dec.pdf')

In [ ]:
plt.ylabel('Intensity')
plt.xlabel('Time (min)')
plt.plot((venus.t - venus.t[0]) / 60, venus.data[3,6])
plt.savefig('timestream.pdf')

In [ ]:
venus.plot_map(3, 6)